<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/Working_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%capture
! apt-get install libboost-python1.71-dev openbabel libopenbabel-dev python-dev zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev
!rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!make clean
!make -j4

In [27]:
from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *
from pyMcEntropy import *
#from pySA import *
#from pyMC import *


In [28]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 30.0 #@param {type:"number"}
Input.search_box_y = 30.0 #@param {type:"number"}
Input.search_box_z = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.3 #@param {type:"number"}
Input.x_dim = 8.0 #@param {type:"number"}
Input.y_dim = 8.0 #@param {type:"number"}
Input.z_dim = 8.0 #@param {type:"number"}
Input.write_grids = True;

In [ ]:

Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
lig = Mol2(Input, "test/lig.mol2.gz")
rec = Mol2(Input, "test/rec.mol2.gz")
Energy = Energy2(Input)

for i in range(len(list(rec.residue_pointer))-1):
  HB.parse_residue(rec.residue_pointer[i]-1, rec.residue_pointer[i+1]-2, rec.resnames[i], rec, lig, 9.0)

HB.find_ligandHB("test/lig.mol2.gz", lig)

com = Coord.compute_com(lig)

print(list(lig.xyz))

energy1 = Energy.compute_ene(rec, lig, lig.xyz)
print("Original energy: %7.3f" % energy1)

Input.write_grids = True;
Grids = Grid(Input, Writer, rec, com)

energy2 = Energy.compute_ene(Grids, lig, lig.xyz)
print("Grid energy: %7.3f" % energy2)